# Emotional Sentiment through Mouse and Keyboard Dynamics
Ryan J Meneses, 2023

## Description
chillpill is an application that collects mouse and keyboard dynamics data from a user and uses it to predict their emotional state. This notebook analyzes the data collected from the application's data collection API and trains linear model to predict the user's emotional state.

In [1]:
import os

from src.util import file

In [2]:
tracking_data = {}
for filename in os.listdir(file.TRACKING_PATH):
    if filename.endswith(".csv"):
        tracking_data[filename] = file.load_tracking_file(filename)
        print(f"Loaded {filename} with {len(tracking_data[filename])} rows.")

tracking_data["20230503-023014.csv"].head()

Loaded 20230503-155648.csv with 100001 rows.
Loaded 20230503-214649.csv with 100000 rows.
Loaded 20230503-023014.csv with 100000 rows.


,id,time,x,y,scroll,press,release,feeling
0,1,1.683096e+09,0.000000,0.000000,0,0,50,1
1,2,1.683096e+09,379.273438,362.273438,0,0,0,1
2,3,1.683096e+09,381.156250,362.273438,0,0,0,1
3,4,1.683096e+09,383.039062,362.273438,0,0,0,1
4,5,1.683096e+09,385.894531,362.273438,0,0,0,1
